In [ ]:
! pip install nltk

In [ ]:
# GET LANGUAGE DETECTION 
!pip install langdetect

     |████████████████████████████████| 983kB 2.8MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=5de5adacec276d1b83d7d320b9ef2a3d60cc29f29cb350fb643c93e9daab5d72
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
import json
import sys, argparse
from scipy import sparse
from sklearn import linear_model
from collections import Counter
import numpy as np
import re
from collections import Counter
from collections import defaultdict
import random
import pandas as pd

from langdetect import detect

import nltk
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('popular')

import string

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [ ]:
yelp = "/content/drive/My Drive/cs182proj/cs182 project/CS182-Spring2020-NLP-Project/yelp_review_training_dataset.jsonl"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
class Prepare:
  data = []

  def __init__(self, path):
    self.path = path
    self.data = []
    self.processed = False
    self.data_added = False
    self.form_extracted = False
    #keep original data for reset change purposes
    self.original = []
    
  # Peak at reviews of certain range to inspect data
  def peak_reviews(self, start, num):
    reviews = self.data
    if self.data_added:
      return [item[1] for item in reviews[start:start+num]]
    else:
      return "Data is not uploaded yet. Call load_data() method first."

  # reset changes in processing by going back to the original reviews
  def reset_reviews(self):
    self.data = self.original
    return "Data is reset to original"

  # Read data from file
  def load_data(self):
    reviews = []
    path = self.path
    with open(path, 'r') as json_file:
      json_list = list(json_file)

    # limit the data loaded by putting json_list[:50] or something here. Don't forget to switch it back
    for json_str in json_list:
      item = json.loads(json_str)
      review = [item["stars"], item["text"]]
      reviews.append(review)

    self.original = reviews
    self.data = reviews
    self.data_added = True
    
    return "Reviews loaded successfully."
    
  # Read dataset and returned featurized representation as sparse matrix + label array
  def process(self):
    if not self.data_added:
      return "Data not uploaded yet. Call load_data() method first."
    reviews = self.data
    for i in range(len(self.data)):
      self.data[i][1] = clean_text(self.data[i][1])
      self.data[i][1] = extra_clean_text(self.data[i][1])
    self.processed = True
    return self.data
  

  # Stem words: remove affixes
  def stem(self):
    if not self.processed:
      self.process()

    reviews = self.data
    if not self.form_extracted:
      porter = PorterStemmer()
      for i in range(len(reviews)):
        review = reviews[i][1]
        reviews[i][1] = [porter.stem(word) for word in review]
      self.form_extracted = True
    self.data = reviews

  # Remove the stop words(asumes you have finish stemming)
  def remove_stopWords(self):  
    if not self.processed:
      self.process()

    if self.form_extracted:
      print("You already finish this step!")
      return
    stopwords_set = {'a','about','after','ain','am','an','and','any','are','as',
                'at','be','been','being','between','both','but','by','can','d',
                'did','do','does','doesn','doing','during','each','few','for',
                'from','had','has','have','having','he','her','here','hers',
                'herself','him','himself','his','how','i','if','in','into','is',
                'it','its','itself','just','m','ma','me','more','most','my',
                'myself','needn',"neednt",'now','o','of','off',
                'on','once','only','or','other','our','ours','ourselves','own',
                're','s','same','shan',"shant",'she',"she's",'should',"shouldve",
                'shouldn',"shouldnt",'so','some','such','t','than','that',
                "thatll",'the','their','theirs','them','themselves','then',
                'there','these','they','this','those','through','to','too',
                'under','until','ve','was','we','were','what','when','where',
                'which','while','who','whom','why','will','with','won','y',
                'you', "youd", "youll", "youre", "youve",'your','yours',
                'yourself','yourselves'}

    reviews = self.data
    for i in range(len(reviews)):
      review = word_tokenize(reviews[i][1])
      reviews[i][1] = [w for w in review if not w in stopwords_set]
    self.data = reviews

  def small_sample(self):
    reviews = self.data
    return random.sample(reviews, 100000)

In [ ]:
### ADD ALL THE HELPER METHODS TO PREPROCESS DATA BELOW HERE, THEN CALL THEM FROM THE "PROCESS" FUNCTION TO USE THEM
### CAN USE PEAK_REVIEWS FUNCTION TO SEE HOW REVIEWS CHANGE

In [ ]:
#convert all apostrope negations is, are, would etc into separate words:
def clean_text(text):
  text = re.sub(r'(is\s*n\s*\'*t)', 'is not ', text,flags=re.IGNORECASE)
  text = re.sub(r'(I\s*\'\s*m)', 'I am ', text,flags=re.IGNORECASE)
  text = re.sub(r'you\s*\'\s*re', 'you are ', text, flags=re.IGNORECASE)
  text =re.sub(r'he\s*\'\s*s', 'he is ', text,flags=re.IGNORECASE)
  text =re.sub(r'she\s*\'\s*s', 'she is ', text,flags=re.IGNORECASE)
  text =re.sub(r'we\s*\'\s*re', 'we are ', text,flags=re.IGNORECASE)
  text =re.sub(r'(it|It)\'\s*s', 'it is ', text,flags=re.IGNORECASE)
  text =re.sub(r'(what|What)\s*\'\s*s', 'what is ', text,flags=re.IGNORECASE)
  text =re.sub(r'are\s*n\s*\'\s*t', 'are not ', text,flags=re.IGNORECASE)
  text =re.sub(r'they\s*\'\s*re', 'they are ', text,flags=re.IGNORECASE)
  text =re.sub(r'there\s*\'\s*s', 'there is ', text,flags=re.IGNORECASE)
  text =re.sub(r'was\s*n\s*\'\s*t', 'was not ', text,flags=re.IGNORECASE)
  text =re.sub(r'were\s*n\s*\'\s*t', 'were not ', text,flags=re.IGNORECASE)           
  text =re.sub(r'I\s*\'\s*ve', 'I have ', text,flags=re.IGNORECASE)
  text =re.sub(r'you\s*\'\s*ve', 'you have ', text,flags=re.IGNORECASE)
  text =re.sub(r'we\s*\'\s*ve', 'we have ', text,flags=re.IGNORECASE)
  text =re.sub(r'they\s*\'\s*ve', 'they have ', text,flags=re.IGNORECASE)
  text =re.sub(r'has\s*n\s*\'\s*t', 'has not ', text,flags=re.IGNORECASE)
  text =re.sub(r'have\s*n\s*\'\s*t', 'have not ', text,flags=re.IGNORECASE) 
  text =re.sub(r'I\s*\'\s*d', 'I had ', text,flags=re.IGNORECASE)
  text =re.sub(r'you\s*\'\s*d', 'you had ', text,flags=re.IGNORECASE)
  text =re.sub(r'he\s*\'\s*d', 'he had ', text,flags=re.IGNORECASE)
  text =re.sub(r'she\s*\'\s*d', 'she had ', text,flags=re.IGNORECASE)
  text =re.sub(r'it\s*\'\s*d', 'it had ', text,flags=re.IGNORECASE)
  text =re.sub(r'we\s*\'\s*d', 'we had ', text,flags=re.IGNORECASE)
  text =re.sub(r'they\s*\'\s*d', 'they had ', text,flags=re.IGNORECASE)
  text =re.sub(r'does\s*n\s*\'\s*t', 'does not ', text,flags=re.IGNORECASE) 
  text =re.sub(r'do\s*n\s*\'\s*t', 'do not ', text,flags=re.IGNORECASE)
  text =re.sub(r'did\s*n\s*\'\s*t', 'did not ', text,flags=re.IGNORECASE)
  text =re.sub(r'I\s*\'\s*ll', 'I will ', text,flags=re.IGNORECASE)
  text =re.sub(r'you\s*\'\s*ll', 'you will ', text,flags=re.IGNORECASE)
  text =re.sub(r'he\s*\'\s*ll', 'he will ', text,flags=re.IGNORECASE)
  text =re.sub(r'she\s*\'\s*ll', 'she will ', text,flags=re.IGNORECASE)
  text =re.sub(r'we\s*\'\s*ll', 'we will ', text,flags=re.IGNORECASE)
  text =re.sub(r'they\s*\'\s*ll', 'they will ', text,flags=re.IGNORECASE)
  text =re.sub(r'there\s*\'\s*ll', 'there will ', text,flags=re.IGNORECASE)
  text =re.sub(r'there\s*\'\s*d', 'there had ', text,flags=re.IGNORECASE)
  text =re.sub(r'ca\s*n\s*\'\s*t', 'can not ', text,flags=re.IGNORECASE)
  text =re.sub(r'can\s*not', 'can not ', text,flags=re.IGNORECASE)
  text =re.sub(r'could\s*n\s*\'\s*t', 'could not ', text,flags=re.IGNORECASE)
  text =re.sub(r'dare\s*n\s*\'\s*t', 'dare not ', text,flags=re.IGNORECASE)
  text =re.sub(r'had\s*n\s*\'\s*t', 'had not ', text,flags=re.IGNORECASE)
  text =re.sub(r'might\s*n\s*\'\s*t', 'might not ', text,flags=re.IGNORECASE)
  text =re.sub(r'must\s*n\s*\'\s*t', 'must not ', text,flags=re.IGNORECASE)
  text =re.sub(r'need\s*n\s*\'\s*t', 'need not ', text,flags=re.IGNORECASE)
  text =re.sub(r'ought\s*n\s*\'\s*t', 'ought not ', text,flags=re.IGNORECASE)
  text =re.sub(r'sha\s*n\s*\'\s*t', 'shall not ', text,flags=re.IGNORECASE)
  text =re.sub(r'should\s*n\s*\'\s*t', 'should not ', text,flags=re.IGNORECASE)
  text =re.sub(r'used\s*n\s*\'\s*t', 'used not ', text,flags=re.IGNORECASE)
  text =re.sub(r'wo\s*n\s*\'\s*t', 'will not ', text,flags=re.IGNORECASE)
  text =re.sub(r'would\s*n\s*\'\s*t', 'would not ', text,flags=re.IGNORECASE)
  text= re.sub(r'b\s*\/\s*c', 'because ', text,flags=re.IGNORECASE)
  text =re.sub(r'\s+\'\s*s', ' is ', text,flags=re.IGNORECASE)
  text =re.sub(r'\s+u\s+', ' you ', text,flags=re.IGNORECASE)
  text =re.sub(r'\s+dr\s+', ' doctor ', text,flags=re.IGNORECASE)
  text = re.sub(r"(w+o+a*w+|w+o+a+h*)", 'wow ', text,flags=re.IGNORECASE)
  text = re.sub(r"sci[\s-]+fi", ' scifi ', text,flags=re.IGNORECASE)
  text = re.sub(r"wanna", ' want to ', text,flags=re.IGNORECASE)
  text = re.sub(r"\s+'\s*re", ' are ', text,flags=re.IGNORECASE)
  text = re.sub(r"\s+'\s*ll", ' will ', text,flags=re.IGNORECASE)
  text = re.sub(r"\s*n\s*\'\s*t", ' not ', text,flags=re.IGNORECASE)
  #remove '\n' and tabs 
  text = re.sub(r"\n", " ",text,flags=re.IGNORECASE)
  text = re.sub(r"\t", " ",text,flags=re.IGNORECASE)
  text = re.sub(r"~", " ",text,flags=re.IGNORECASE)
  #remove hyperlink
  text = re.sub(r"http\S+", "",text,flags=re.IGNORECASE)

  # convert Asli's to Asli has (language ungrammatical but better embedding rep)
  text = re.sub("\'s", " has ", text,flags=re.IGNORECASE)

  # man o'war = man of war
  text = re.sub(r"o\'|O\'", " of ", text,flags=re.IGNORECASE)
  
  return text

In [ ]:
  #lower case, remove digits, remove punctuation
  def extra_clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", "",text)
    text = re.sub(r'[^\w\s]','',text)
    return text  

In [ ]:
  # Remove the stop words(asumes you have finish stemming)
  def remove_stopWords(revs):
    
    stopwords_set = {'a','about','after','ain','am','an','and','any','are','as',
                'at','be','been','being','between','both','but','by','can','d',
                'did','do','does','doesn','doing','during','each','few','for',
                'from','had','has','have','having','he','her','here','hers',
                'herself','him','himself','his','how','i','if','in','into','is',
                'it','its','itself','just','m','ma','me','more','most','my',
                'myself','needn',"neednt",'now','o','of','off',
                'on','once','only','or','other','our','ours','ourselves','own',
                're','s','same','shan',"shant",'she',"she's",'should',"shouldve",
                'shouldn',"shouldnt",'so','some','such','t','than','that',
                "thatll",'the','their','theirs','them','themselves','then',
                'there','these','they','thi','those','through','to','too',
                'under','until','ve','was','we','were','what','when','where',
                'which','while','who','whom','why','will','with','won','y',
                'you', "youd", "youll", "youre", "youve",'your','yours',
                'yourself','yourselves'}
    for i in range(len(revs)):
      re = word_tokenize(revs[i][1])
      revs[i][1] = [w for w in re if not w in stopwords_set]
    return revs 

# **Data prepare**

In [ ]:
reviews = Prepare(yelp)
reviews.load_data()

'Reviews loaded successfully.'

In [ ]:
reviews.peak_reviews(0,10)

['Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.',
 "I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains that offer this service; however, Travis has taken the flawless blowout to a whole new level!  \n\nTravis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a Vegas-worthy rockstar outfit.  Next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure even the very worst migraine in minutes --- and the scented shampoo room.  Travis has freakishly strong fingers (in a good way) and use the perfect amount of pressure.  That was superb!  Then starts the glorious blowout... where not one, not two, but THREE people were involved in doing the best round-brush action my hair

#**basic processing**
* lower case, remove digits, punctuation 

In [ ]:
reviews.process()
reviews.peak_reviews(0,10)

['total bill for this horrible service over gs these crooks actually had the nerve to charge us  for  pills i checked online the pills can be had for  cents each avoid hospital ers at all costs',
 'i adore travis at the hard rock has  new kelly cardenas salon  i am  always a fan of a great blowout and no stranger to the chains that offer this service however travis has taken the flawless blowout to a whole new level    travis has  greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a vegasworthy rockstar outfit  next comes the most relaxing and incredible shampoo  where you get a full head message that could cure even the very worst migraine in minutes  and the scented shampoo room  travis has freakishly strong fingers in a good way and use the perfect amount of pressure  that was superb  then starts the glorious blowout where not one not two but three people were involved in doing the best roundbrush action my hair has ever seen  the team of sty

In [ ]:
reviews.peak_reviews(0,20)

['total bill for this horrible service over gs these crooks actually had the nerve to charge us  for  pills i checked online the pills can be had for  cents each avoid hospital ers at all costs',
 'i adore travis at the hard rock has  new kelly cardenas salon  i am  always a fan of a great blowout and no stranger to the chains that offer this service however travis has taken the flawless blowout to a whole new level    travis has  greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a vegasworthy rockstar outfit  next comes the most relaxing and incredible shampoo  where you get a full head message that could cure even the very worst migraine in minutes  and the scented shampoo room  travis has freakishly strong fingers in a good way and use the perfect amount of pressure  that was superb  then starts the glorious blowout where not one not two but three people were involved in doing the best roundbrush action my hair has ever seen  the team of sty

# add UNKLANG token at the beginning of all non-english reviews

In [ ]:
# add UNKLANG token at the beginning of all non-english reviews
for i in range(len(reviews.data)):
  item = reviews.data[i]
  try:
    if detect(item[1])!="en" and reviews.data[i][1].split()[0] != "UNKLANG":
      reviews.data[i][1] = "UNKLANG " + reviews.data[i][1]
      #print(reviews.data[i][1])
  except:
    print("forcing")
    reviews.data[i][1] = "UNKLANG " + reviews.data[i][1]
    print(reviews.data[i][1])

forcing
UNKLANG 
forcing
UNKLANG 
forcing
UNKLANG 
forcing
UNKLANG 
forcing
UNKLANG the best moving experience ever   we have moved twice in the past year and would not  consider any other movers  they are polite fast efficient hard working and the best movers we have ever used  i can not  say enough about all of the employees at eduardo has  they even came on time  we have a lot of very large heavy pieces  a safe treadmill outdoor furniture  to name a few and nothing was damaged or even scratched they are the best
forcing
UNKLANG rude and too much clutter do not  have any prices on braiding hair packages  they charge a different price for braiding hair every time i shop here i am  done no more
forcing
UNKLANG mechanic in my family for about  years christian man with ethics and morals great mechanic i will  drive  miles if i had to to go to rob


In [ ]:
reviews.data[2]

# **Remove stopwords**

In [ ]:
reviews.remove_stopWords()
reviews.peak_reviews(0,10)

In [ ]:
reviews.peak_reviews(1,3)

**save text with stopwords removed**

In [ ]:
import csv

path = '/content/drive/My Drive/cs182proj/NLP data set/'

with open(path+'remove_stopwords.csv', mode='w') as f:
    here = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    here.writerow(["stars", "review"])
    for i in range(len(reviews.data)):
      l = reviews.data[i][1]
      here.writerow([reviews.data[i][0],' '.join(l)])

# **Stemming**

In [ ]:
#assumes tokenized (do this step after removing stopwords)
reviews.stem()

**save the reviews after stemming**

In [ ]:
with open(path+'preprocessed_remove_stopwords_stem.csv', mode='w') as f:
    here = csv.writer(f)

    here.writerow(["stars", "review"])
    for i in range(len(reviews.data)):
      l = reviews.data[i][1]
      here.writerow([reviews.data[i][0],' '.join(l)])

NameError: ignored

#Remove low frequency words

In [ ]:
path = '/content/drive/My Drive/cs182proj/NLP data set/preprocessed_remove_stopwords_stem.csv'
reviews = pd.read_csv(path)

In [ ]:
reviews['review_tokenize'] = reviews['review'].apply(lambda r: word_tokenize(r) if(type(r)!=float) else 'NA')

Get low frequency words

In [ ]:
def get_words_list(reviews):
  words = []
  print(type(reviews[0]))
  for i in range(len(reviews)):
    words.extend(reviews[i])
  return words

words = get_words_list(reviews['review_tokenize'])

<class 'list'>


In [ ]:
words_series = pd.Series(words)
word_count = words_series.value_counts()    

In [ ]:
len(words_series)

28792591

# **Create randomly chosen train, and validation sets as 0.2 and 0.8 of training data**
**can create new random sets by running the function with different names**

In [ ]:

dataset = processed_data[["stars","review"]]
fraction = 0.2
name = "Shixin"

In [ ]:
validation = dataset.sample(frac=fraction, replace=False, random_state=1)
training = dataset.loc[~dataset.index.isin(list(validation.index))]

validation.to_csv("drive/My Drive/CS182/Code/validation_set_" + str(name) + ".csv")
training.to_csv("drive/My Drive/CS182/Code/training_set_" + str(name) + ".csv")

# **small sample**

In [ ]:
small_reviews = reviews.small_sample()

In [ ]:
len(small_reviews)

100000

In [ ]:
small_reviews = np.array(small_reviews)

In [ ]:
def data_to_csv(text, stars):
  df = pd.DataFrame({'text': text, 'stars': stars})
  df.to_csv('small_data.csv')
  !cp small_data.csv "drive/My Drive/"

In [ ]:
data_to_csv(small_reviews[:, 1], small_reviews[:, 0])

# CSV to Array

In [ ]:
small_data = pd.read_csv('drive/My Drive/small_data.csv').iloc[:,1:3].values  

In [ ]:
small_data[0:2]

NameError: ignored

# **language dectector:** https://github.com/Mimino666/langdetect


## ***Naive Bayes Classification Model based on n-gram information using smoothing ⇒ rating prediction***
<br>

1. No extra representation here, literally count unigram (bag of words),     bigram, trigram words by counting each of their occurences in review

2. Use some sort of smoothing method of your choice (otherwise a bunch of new bigrams or trigrams will be unseen at test time)
  * Naive adding
  * Kneser-Ney Smoothing
  * Stupid Backoff (I swear this is the actual name) method

3. Use Naive Bayes classification to predict rating information
  * The n-grams therefore should be counted with respect to the review’s rating during training so the probabilistic model works
<br>


**Naive Bayes principle**: 
https://blog.csdn.net/qiu_zhi_liao/article/details/90671932
<br />

**N-Gram model:**
https://wizardforcel.gitbooks.io/learn-dl-with-pytorch-liaoxingyu/5.4.2.html

**Naive Bayes Classifier for n-gram**
https://stackoverflow.com/questions/48003907/how-to-train-naive-bayes-classifier-for-n-gram-movie-reviews

**Naive Bayes Classifier code**
https://www.nltk.org/_modules/nltk/classify/naivebayes.html
